In [1]:
from utils_v1 import *


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates,
    PA, PADeforestationTime, PADistanceMap, PAMultipleDates,
    MTDeforestationTime,
    MTMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential

from src.trainer.evidential import TrainerEvidential

from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.trainer.evidential as evidential


In [3]:

config = {
	"training": True,
	"learning_rate": 1e-4,
	"weights": [0.1, 0.9, 0],
	"save_probabilities": False,
	"loadInference": False,
	"addPastDeforestationInput": True,
	"inference_times": 1,
	"Uncertainty": {
		"annealing_step": 10*375/2 # 1875
	}
}




'''
if config["training"] == True:
	dates = [2017, 2018]
else:
	dates = [2018, 2019]
if config["training"] == True:
	dates = [2018, 2019]
else:
	dates = [2019, 2020]
'''
# dates = [2018, 2019]
# dates = [2017, 2018]

# dataset = PAMultipleDates(dates = dates, # dates = [2017, 2018, 2019]. [2018, 2019]. [2015, 2016, 2017, 2018]
# 	addPastDeforestationInput = config['addPastDeforestationInput'],
# 	borderBuffer = 2)	
dates = [2018, 2019]
# dates = [2019, 2020]

dataset = PAMultipleDates(dates = dates, # dates = [2016,2017,2018,2019]
	addPastDeforestationInput = config['addPastDeforestationInput'],
	borderBuffer = 2)	
# dataset = MTDeforestationTime()	
# [2016, 2017, 2018]

ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerEvidential(
		dataset)
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [4]:
[10*375//1, 10*375//1.5, 10*375//2, 10*375//3, 10*375//4]


[3750, 2500.0, 1875, 1250, 937]

In [5]:
# if config["training"] == True:
# experiment_values = [10*375//1, 10*375//1.5, 10*375//2, 10*375//3, 10*375//4]
# experiment_values = [10*375//3, 10*375//3, 10*375//3, 10*375//3, 10*375//3]
# experiment_values = [10*375//2, 10*375//2, 10*375//2, 10*375//2, 10*375//2, 10*375//2, 10*375//2]
# experiment_values = [10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375]
experiment_values = [10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375]

experiment_values = [int(x) for x in experiment_values]
results = []

#exp_id_base = 148
# exp_id_base = 155
exp_id_base = 185


exp_ids = range(exp_id_base, exp_id_base+len(experiment_values))


In [6]:
exp_ids

range(175, 185)

In [7]:
from enum import Enum
UncertaintyMethod = Enum('Predictor', 'pred_var MI pred_entropy KL pred_entropy_single evidential')
uncertaintyMethod = "evidential"


In [8]:


for experiment_value, exp_id in zip(experiment_values, exp_ids):
    config = {
        "training": True,
    	"learning_rate": 1e-4,
        "weights": [0.1, 0.9, 0],
        "save_probabilities": False,
        "loadInference": False,
        "addPastDeforestationInput": True,
        "inference_times": 1,
        "Uncertainty": {
            "annealing_step": 10*375/2 # 1875
        },
    	"uncertainty_method": uncertaintyMethod,
        "evidence_function": evidential.relu_evidence
    }

    trainer = TrainerEvidential(config, dataset, patchesHandler)

    trainer.defineExperiment(exp_id)

    trainer.setExperimentPath()
    trainer.createLogFolders()

    trainer.loadDataset()

    # %%
    if config["training"] == True:
        trainer.run()

    # %%
    result = trainer.run_predictor()
    results.append(result)


    with open('log.pkl', 'wb') as f:
        pickle.dump(results, f)

D:/Jorge/datasets/deforestation/MT/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([161124576,   1332784,    240796,    338545,    313901,    287191,
                                                                       708581,    648966,    876898,   1007787,   1098901,   1054092,
                                                                      2096062,   2473828,   1400992], dtype=int64))


Label where deforestation past years is actual date (2019) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([161124576,   1332784,    240796,    338545,    313901,    287191,
                                                                       708581,    648966,    876898,   1007787,   1098901,   1054092,
                                                                      2096062,   2473828,   1400992], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

cm_correct [[37759929   339416]
 [   12385   431088]]
cm_incorrect [[1012658   90988]
 [  60894   27521]]
[0.55948833 0.97207271 0.81048642 0.03000037 0.23222709 0.31127071]
threshold 0.5385301734249958
(38542818,) (38542818,)


ic| label_current_deforestation_test_classified_incorrect.shape: (1192061,)
    predicted_test_classified_incorrect.shape: (1192061,)
ic| TP_H + FN_H + FP_H + TN_H: 1192061
    len(label_mask_current_deforestation_test): 39734879
ic| TP_H + FN_H + FP_H + TN_H: 1192061
    len(label_mask_current_deforestation_test): 39734879


cm_correct [[37759929   339416]
 [   12385   431088]]
cm_incorrect [[1012658   90988]
 [  60894   27521]]
[0.55948833 0.97207271 0.81048642 0.03000037 0.23222709 0.31127071]
threshold 0.5385301734249958
(39734879,) (39734879,)


ic| self.m_optimal: {'AA': array([0.03000037]),
                     'f1': 64.55,
                     'f1_H': array([0.26600104]),
                     'f1_L': array([0.71020785]),
                     'precision_H': array([0.23222709]),
                     'precision_L': array([0.55948833]),
                     'recall_H': array([0.31127071]),
                     'recall_L': array([0.97207271]),
                     'recall_Ltotal': array([0.81048642])}
ic| self.m_audited_optimal: {'f1': array([0.74705261]),
                             'precision': array([0.60483352]),
                             'recall': array([0.97671502])}


cm_audited [[38863575   339416]
 [   12385   519503]]
[0.60483352 0.97671502]


In [9]:
print(results)

[{'metrics': {'precision_L': array([nan]), 'recall_L': array([0.]), 'recall_Ltotal': array([0.]), 'AA': array([0.03000046]), 'precision_H': array([nan]), 'recall_H': array([0.]), 'f1_L': array([nan]), 'f1_H': array([nan]), 'f1': 0.0}, 'metrics_audited': {'precision': array([1.]), 'recall': array([0.88574556]), 'f1': array([0.93941153])}, 'exp': 175}, {'metrics': {'precision_L': array([0.53704756]), 'recall_L': array([0.9351188]), 'recall_Ltotal': array([0.85489489]), 'AA': array([0.02999747]), 'precision_H': array([0.16052591]), 'recall_H': array([0.41546518]), 'f1_L': array([0.68226429]), 'f1_H': array([0.23157625]), 'f1': 63.3}, 'metrics_audited': {'precision': array([0.56072183]), 'recall': array([0.94068496]), 'f1': array([0.70262449])}, 'exp': 176}, {'metrics': {'precision_L': array([0.70278881]), 'recall_L': array([0.93463832]), 'recall_Ltotal': array([0.76870477]), 'AA': array([0.02999925]), 'precision_H': array([0.36952487]), 'recall_H': array([0.37348836]), 'f1_L': array([0.80

In [12]:
print(len(results))

10


In [1]:
results[9]

NameError: name 'results' is not defined

In [10]:
trainer.f1

64.55

In [11]:
print(trainer.f1)

64.55
